# Прогноз спроса

### Обработка данных 

In [111]:
#все импорты
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [112]:
df = pd.read_excel('Обновленный_Отчет.xlsx')
df

,Компания,Продукт,Дата,Сумма (с НДС),Количество
0,Компания 17,"Профнастил ""Stalcolor""",17.01.2020,9.05,1.99
1,Компания 23,"Профнастил ""Stalcolor""",23.01.2020,271.55,59.55
2,Компания 5,"Профнастил ""Stalcolor""",22.01.2020,228.67,47.64
3,Компания 57,Сайдинг,31.01.2020,54.31,11.91
4,Компания 46,Сайдинг,27.01.2020,25.01,5.96
...,...,...,...,...,...
21170,Компания 29,"Плоский лист ""Stalcolor""",14.07.2025,353.27,13.86
21171,Компания 35,Сайдинг,17.07.2025,2544.72,99.84
21172,Компания 16,Металлочерепица Stalcolor,18.07.2025,1029.52,40.39
21173,Компания 58,"Профнастил ""Stalcolor""",31.07.2025,1403.88,55.08


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21175 entries, 0 to 21174
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Компания       21175 non-null  object 
 1   Продукт        21175 non-null  object 
 2   Дата           21175 non-null  object 
 3   Сумма (с НДС)  21175 non-null  float64
 4   Количество     21175 non-null  float64
dtypes: float64(2), object(3)
memory usage: 827.3+ KB


In [114]:
print("Дубликатов:", df.duplicated().sum())

Дубликатов: 0


In [115]:
df.isnull().sum()

Компания         0
Продукт          0
Дата             0
Сумма (с НДС)    0
Количество       0
dtype: int64

In [116]:
df.columns = [c.strip().lower() for c in df.columns]

In [117]:
# Преобразуем дату
df['дата'] = pd.to_datetime(df['дата'])
df

,компания,продукт,дата,сумма (с ндс),количество
0,Компания 17,"Профнастил ""Stalcolor""",2020-01-17,9.05,1.99
1,Компания 23,"Профнастил ""Stalcolor""",2020-01-23,271.55,59.55
2,Компания 5,"Профнастил ""Stalcolor""",2020-01-22,228.67,47.64
3,Компания 57,Сайдинг,2020-01-31,54.31,11.91
4,Компания 46,Сайдинг,2020-01-27,25.01,5.96
...,...,...,...,...,...
21170,Компания 29,"Плоский лист ""Stalcolor""",2025-07-14,353.27,13.86
21171,Компания 35,Сайдинг,2025-07-17,2544.72,99.84
21172,Компания 16,Металлочерепица Stalcolor,2025-07-18,1029.52,40.39
21173,Компания 58,"Профнастил ""Stalcolor""",2025-07-31,1403.88,55.08


In [118]:
df['количество']=pd.to_numeric(df['количество'])
df

,компания,продукт,дата,сумма (с ндс),количество
0,Компания 17,"Профнастил ""Stalcolor""",2020-01-17,9.05,1.99
1,Компания 23,"Профнастил ""Stalcolor""",2020-01-23,271.55,59.55
2,Компания 5,"Профнастил ""Stalcolor""",2020-01-22,228.67,47.64
3,Компания 57,Сайдинг,2020-01-31,54.31,11.91
4,Компания 46,Сайдинг,2020-01-27,25.01,5.96
...,...,...,...,...,...
21170,Компания 29,"Плоский лист ""Stalcolor""",2025-07-14,353.27,13.86
21171,Компания 35,Сайдинг,2025-07-17,2544.72,99.84
21172,Компания 16,Металлочерепица Stalcolor,2025-07-18,1029.52,40.39
21173,Компания 58,"Профнастил ""Stalcolor""",2025-07-31,1403.88,55.08


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21175 entries, 0 to 21174
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   компания       21175 non-null  object        
 1   продукт        21175 non-null  object        
 2   дата           21175 non-null  datetime64[ns]
 3   сумма (с ндс)  21175 non-null  float64       
 4   количество     21175 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 827.3+ KB


In [120]:
# === 2. Признаки для прогноза спроса ===
# Добавим признаки даты
df['месяц'] = df['дата'].dt.month
df['квартал'] = df['дата'].dt.quarter
df['год'] = df['дата'].dt.year
df['день_недели'] = df['дата'].dt.weekday
df

,компания,продукт,дата,сумма (с ндс),количество,месяц,квартал,год,день_недели
0,Компания 17,"Профнастил ""Stalcolor""",2020-01-17,9.05,1.99,1,1,2020,4
1,Компания 23,"Профнастил ""Stalcolor""",2020-01-23,271.55,59.55,1,1,2020,3
2,Компания 5,"Профнастил ""Stalcolor""",2020-01-22,228.67,47.64,1,1,2020,2
3,Компания 57,Сайдинг,2020-01-31,54.31,11.91,1,1,2020,4
4,Компания 46,Сайдинг,2020-01-27,25.01,5.96,1,1,2020,0
...,...,...,...,...,...,...,...,...,...
21170,Компания 29,"Плоский лист ""Stalcolor""",2025-07-14,353.27,13.86,7,3,2025,0
21171,Компания 35,Сайдинг,2025-07-17,2544.72,99.84,7,3,2025,3
21172,Компания 16,Металлочерепица Stalcolor,2025-07-18,1029.52,40.39,7,3,2025,4
21173,Компания 58,"Профнастил ""Stalcolor""",2025-07-31,1403.88,55.08,7,3,2025,3


In [121]:
# Средняя цена
df['средняя_цена'] = df['сумма (с ндс)'] / df['количество']
df.head()

,компания,продукт,дата,сумма (с ндс),количество,месяц,квартал,год,день_недели,средняя_цена
0,Компания 17,"Профнастил ""Stalcolor""",2020-01-17,9.05,1.99,1,1,2020,4,4.547739
1,Компания 23,"Профнастил ""Stalcolor""",2020-01-23,271.55,59.55,1,1,2020,3,4.560034
2,Компания 5,"Профнастил ""Stalcolor""",2020-01-22,228.67,47.64,1,1,2020,2,4.799958
3,Компания 57,Сайдинг,2020-01-31,54.31,11.91,1,1,2020,4,4.560034
4,Компания 46,Сайдинг,2020-01-27,25.01,5.96,1,1,2020,0,4.196309


In [122]:
# Агрегация на уровне Компания+Продукт+Месяц
features_demand = df.groupby(['компания', 'продукт', 'год', 'месяц']).agg({
'сумма (с ндс)': ['sum', 'mean'],
'количество': ['sum', 'mean'],
'средняя_цена': 'mean',
'день_недели': 'nunique'
}).reset_index()
features_demand 

компания                        продукт   год месяц сумма (с ндс)  \
                                                                      sum   
0     Компания 1  Конек полукруглый "Stalcolor"  2020     3       3287.62   
1     Компания 1  Конек полукруглый "Stalcolor"  2020     5        121.97   
2     Компания 1  Конек полукруглый "Stalcolor"  2021     1       1599.59   
3     Компания 1  Конек полукруглый "Stalcolor"  2021     2       1709.01   
4     Компания 1  Конек полукруглый "Stalcolor"  2021     3        356.95   
...          ...                            ...   ...   ...           ...   
9736  Компания 9                         Штрипс  2021    11        645.53   
9737  Компания 9                         Штрипс  2021    12        436.80   
9738  Компания 9                         Штрипс  2024     6        764.87   
9739  Компания 9                         Штрипс  2025     2       2934.96   
9740  Компания 9                         Штрипс  2025     7        283.07   

               количество           средняя_цена день_недели  
          mean        sum      mean         mean     nunique  
0     1643.810     473.40   236.700     6.378008           2  
1      121.970      61.60    61.600     1.980032           1  
2      319.918     987.57   197.514     1.309173           3  
3      854.505    1128.22   564.110     6.342544           1  
4      356.950     297.46   297.460     1.199993           1  
...        ...        ...       ...          ...         ...  
9736   322.765     473.92   236.960     1.220992           2  
9737   436.800    2153.78  2153.780     0.202806           1  
9738   764.870      29.24    29.240    26.158345           1  
9739  2934.960     100.32   100.320    29.255981           1  
9740   283.070      13.46    13.460    21.030461           1  

[9741 rows x 10 columns]

In [123]:
# Упростим имена колонок
features_demand.columns = ['компания', 'продукт', 'год', 'месяц',
'сумма_общая', 'сумма_средняя',
'количество_общее', 'количество_среднее',
'средняя_цена', 'уникальные_дни']
features_demand

,компания,продукт,год,месяц,сумма_общая,сумма_средняя,количество_общее,количество_среднее,средняя_цена,уникальные_дни
0,Компания 1,"Конек полукруглый ""Stalcolor""",2020,3,3287.62,1643.810,473.40,236.700,6.378008,2
1,Компания 1,"Конек полукруглый ""Stalcolor""",2020,5,121.97,121.970,61.60,61.600,1.980032,1
2,Компания 1,"Конек полукруглый ""Stalcolor""",2021,1,1599.59,319.918,987.57,197.514,1.309173,3
3,Компания 1,"Конек полукруглый ""Stalcolor""",2021,2,1709.01,854.505,1128.22,564.110,6.342544,1
4,Компания 1,"Конек полукруглый ""Stalcolor""",2021,3,356.95,356.950,297.46,297.460,1.199993,1
...,...,...,...,...,...,...,...,...,...,...
9736,Компания 9,Штрипс,2021,11,645.53,322.765,473.92,236.960,1.220992,2
9737,Компания 9,Штрипс,2021,12,436.80,436.800,2153.78,2153.780,0.202806,1
9738,Компания 9,Штрипс,2024,6,764.87,764.870,29.24,29.240,26.158345,1
9739,Компания 9,Штрипс,2025,2,2934.96,2934.960,100.32,100.320,29.255981,1


In [124]:
y = features_demand[['количество_общее']]
X = features_demand.drop(columns=['компания', 'продукт', 'количество_общее'])

In [125]:
# Кодируем категориальные признаки (год, месяц уже числовые)
X = pd.get_dummies(X, columns=['год', 'месяц'])

In [126]:
X

,сумма_общая,сумма_средняя,количество_среднее,средняя_цена,уникальные_дни,год_2020,год_2021,год_2022,год_2023,год_2024,...,месяц_3,месяц_4,месяц_5,месяц_6,месяц_7,месяц_8,месяц_9,месяц_10,месяц_11,месяц_12
0,3287.62,1643.810,236.700,6.378008,2,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,121.97,121.970,61.600,1.980032,1,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1599.59,319.918,197.514,1.309173,3,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1709.01,854.505,564.110,6.342544,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,356.95,356.950,297.460,1.199993,1,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9736,645.53,322.765,236.960,1.220992,2,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9737,436.80,436.800,2153.780,0.202806,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
9738,764.87,764.870,29.240,26.158345,1,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,False
9739,2934.96,2934.960,100.320,29.255981,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [152]:
# Initial split into training and temporary set
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [154]:
print(f"Train:{len(X_train)}, Validation: {len(X_val)}, Test:{len(X_test)}")

Train:7792, Validation: 974, Test:975


## Линейная регрессия sklearn

In [130]:
from sklearn.linear_model import LinearRegression

In [131]:
model_skl = LinearRegression()
model_skl.fit(X_train, y_train)

LinearRegression()

In [132]:
model_skl.coef_

array([[ 4.98797719e-01, -7.57061246e-01,  2.15460789e+00,
         1.95481022e-01,  5.59716873e+02, -1.53876508e+02,
        -4.12949527e+02,  1.34574093e+02,  1.03942897e+02,
         1.98561496e+02,  1.29747549e+02,  7.70452320e+01,
        -8.75286900e+01, -1.63594794e+02, -2.43537775e+02,
        -5.84892270e+01,  1.55953110e+02,  5.57233569e+02,
        -3.41023901e+01, -7.93767667e+01, -9.44608339e+01,
        -1.74940019e+01, -1.16474325e+01]])

In [133]:
model_skl.intercept_

array([-554.45896752])

In [134]:
y_train_pred_skl = model_skl.predict(X_train)
y_train_pred_skl

array([[ 941.89424772],
       [-278.18982686],
       [4974.75369507],
       ...,
       [ 123.13985822],
       [2167.49372214],
       [ -57.24292471]], shape=(7792, 1))

In [135]:
y_val_pred_skl = model_skl.predict(X_val)
y_val_pred_skl

array([[ 2.50550242e+03],
       [ 3.16163347e+03],
       [ 4.92807572e+02],
       [ 4.79856326e+01],
       [ 1.91750841e+03],
       [ 8.48192413e+01],
       [ 3.47537660e+03],
       [-5.29045892e+01],
       [ 1.68063683e+03],
       [ 2.21781569e+03],
       [ 2.27072183e+03],
       [-1.61444590e+02],
       [ 1.31206600e+01],
       [ 8.29169573e+01],
       [ 1.62472722e+02],
       [-3.50798511e+02],
       [ 2.45068649e+01],
       [ 1.40218886e+03],
       [ 3.08433055e+01],
       [ 4.37380341e+03],
       [ 1.03637570e+04],
       [-3.72067494e+02],
       [ 9.50537833e+02],
       [ 7.21098634e+03],
       [-2.22322414e+02],
       [ 1.47618229e+02],
       [ 4.55481784e+03],
       [-1.67411086e+01],
       [-3.17806107e+02],
       [ 5.84240546e+03],
       [ 7.29985050e+02],
       [-2.40590790e+02],
       [ 1.13801739e+02],
       [ 8.91017414e+02],
       [ 1.12533995e+03],
       [-3.11928314e+02],
       [ 3.83111707e+01],
       [ 6.21104546e+02],
       [ 2.7

### Метрики качества регрессии (хуйня пиздец)

In [136]:
from sklearn import metrics

In [137]:
MSE_val_model_skl = metrics.mean_squared_error (y_val, y_val_pred_skl)
MSE_val_model_skl

1387204.7275302354

In [138]:
RMSE_train_model_skl = metrics. root_mean_squared_error (y_train, y_train_pred_skl)
RMSE_train_model_skl

1258.682742121594

In [139]:
RMSE_val_model_skl = metrics.root_mean_squared_error (y_val, y_val_pred_skl)
RMSE_val_model_skl

1177.796556086931

In [140]:
MAE_train_model_skl = metrics.mean_absolute_error (y_train, y_train_pred_skl)
MAE_train_model_skl

661.2469792223736

In [141]:
MAE_val_model_skl = metrics.mean_absolute_error (y_val, y_val_pred_skl)
MAE_val_model_skl

673.3265381184539

In [142]:
MAPE_train_model_skl = metrics.mean_absolute_percentage_error (y_train, y_train_pred_skl)
MAPE_train_model_skl

8.538693754103637

In [143]:
MAPE_val_model_skl = metrics.mean_absolute_percentage_error (y_val, y_val_pred_skl)
MAPE_val_model_skl

7.781769841916076

### Бустинги

In [144]:
import lightgbm as lgbm

In [145]:
model = lgbm.LGBMRegressor()
model.fit(X_train, y_train)

LGBMRegressor()

In [146]:
y_val_pred = model.predict(X_val)

In [147]:
mae_val = mean_absolute_error(y_val, y_val_pred)
mape_val = mean_absolute_percentage_error(y_val, y_val_pred)
print(f'Validation MAE: {mae_val}')
print(f'Validation MAPE: {mape_val * 100:.2f}%')

Validation MAE: 179.74554506559437
Validation MAPE: 43.68%


In [148]:
y_test_pred = model.predict(X_test)

In [149]:
mae_test = mean_absolute_error(y_test, y_test_pred)
mape_test = mean_absolute_percentage_error(y_test, y_test_pred)
print(f'Test MAE: {mae_test}')
print(f'Test MAPE: {mape_test * 100:.2f}%')

Test MAE: 187.19939768277166
Test MAPE: 49.98%


In [156]:
import optuna

In [157]:
def objective(trial):
    params = {
        'objective': 'regression',
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),  # Оптимизируем num_leaves
        'max_depth': trial.suggest_int('max_depth', -1, 10),  # -1 для неограниченной глубины
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # Оптимизируем
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # Оптимизируем
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1.0),  # Оптимизируем
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 1.0),
    }


    model_2 = lgbm.LGBMRegressor(**params)
    model_2.fit(X_train, y_train)

    # Предсказание на валидационной выборке
    y_val_pred = model_2.predict(X_val)
    
    # Вычисление MAPE и MAE
    mape = mean_absolute_percentage_error(y_val, y_val_pred)

    return mape

In [158]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1500)

# Вывод лучших параметров и значений MAPE и MAE
best_params = study.best_params
best_mape = study.best_value * 100  # Переводим в проценты

[I 2025-09-25 20:10:10,813] A new study created in memory with name: no-name-cc35748e-f0b9-4470-83a9-1ff6d9747d23
[I 2025-09-25 20:10:11,452] Trial 0 finished with value: 22.322293608383163 and parameters: {'num_leaves': 73, 'max_depth': 5, 'learning_rate': 0.007686183166329303, 'n_estimators': 115, 'subsample': 0.8897046190953923, 'colsample_bytree': 0.7907769041145633, 'reg_alpha': 0.511980300598986, 'lambda_l1': 0.41955879728611056, 'lambda_l2': 0.005149655647325208}. Best is trial 0 with value: 22.322293608383163.
[I 2025-09-25 20:10:11,991] Trial 1 finished with value: 4.990315185922724 and parameters: {'num_leaves': 50, 'max_depth': 8, 'learning_rate': 0.01457937070306385, 'n_estimators': 172, 'subsample': 0.5470375360434399, 'colsample_bytree': 0.8734500049541798, 'reg_alpha': 0.03720017947662924, 'lambda_l1': 0.019674263005640034, 'lambda_l2': 0.10697774453120176}. Best is trial 1 with value: 4.990315185922724.
[I 2025-09-25 20:10:12,658] Trial 2 finished with value: 26.6198425

In [160]:
# Best parameters and evaluation
print("Best hyperparameters:", study.best_params)
print(f"Best MAPE: {study.best_value*100:.2f}%")

Best hyperparameters: {'num_leaves': 137, 'max_depth': -1, 'learning_rate': 0.10215477838098323, 'n_estimators': 178, 'subsample': 0.6557604777312063, 'colsample_bytree': 0.9794586849380688, 'reg_alpha': 0.09170799183275406, 'lambda_l1': 0.7880804131157007, 'lambda_l2': 0.8145349274252128}
Best MAPE: 9.71%


In [161]:
best_model = lgbm.LGBMRegressor(**best_params)
best_model.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.9885208859139814, lambda_l1=0.5371294169434199,
              lambda_l2=0.001998456635362174, learning_rate=0.11103657846953723,
              n_estimators=168, num_leaves=150, reg_alpha=0.03760135582087517,
              subsample=0.9230528085708893)

In [162]:
y_val_pred = best_model.predict(X_val)
best_mae = mean_absolute_error(y_val, y_val_pred)

print(f"Best MAPE: {best_mape:.2f}%")  # Выводим MAPE в процентах
print(f"Best MAE: {best_mae:.2f}")

Best MAPE: 8.42%
Best MAE: 165.18


In [163]:
y_test_pred = best_model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

print(f'Test MAE: {test_mae}')
print(f'Test MAPE: {test_mape * 100:.2f}%')

Test MAE: 163.80791378357472
Test MAPE: 8.79%
